In [1]:
import os,sys
cwd = os.path.abspath(os.path.curdir)
sys.path.append(cwd)  # workplace
import time


import numpy as np
import pandas as pd
import torch

import argparse


In [3]:

def get_dag_table(dataname,seed_sim):
    adj_path = cwd+f'/data/sim_{dataname}/{seed_sim}/generated_graph_target.csv'
    graph_np = pd.read_csv(adj_path)
    graph_np = graph_np.iloc[:10,:10]
    dag_gt = graph_np.to_numpy()

    data_train_path = cwd+f'/data/sim_{dataname}/{seed_sim}/train.csv'
    data_train = pd.read_csv(data_train_path)
    data_train = data_train.iloc[:,:10]
    return dag_gt,data_train  

dag_gt,data_train = get_dag_table('lu',100)
# adj_gt = get_adj(dag_gt) 


In [4]:
dag_gt

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [12]:
heads[0]

3

In [14]:
dag_gt.shape

(10, 10)

In [17]:
heads, tails = np.where(dag_gt == 1)
nrow,ncol = dag_gt.shape
# causal graph nodes
prompt_dag = 'A causal graph has nodes '
for i in range(nrow-1):
    prompt_dag += f'V{i}, '
prompt_dag += f'and V{i+1}. And its edges are '

# causal graph edges
for i in range(len(heads)-1):
    prompt_dag += f'V{heads[i]} -> V{tails[i]}, '
prompt_dag += f'and V{heads[i+1]} -> V{tails[i+1]}. '


# causal graph reasoning questions
prompt_dag += 'What are the neighbors of each node?'
print(prompt_dag)

A causal graph has nodes V0, V1, V2, V3, V4, V5, V6, V7, V8, and V9. And its edges are V3 -> V5, V4 -> V5, V4 -> V9, V6 -> V7, and V6 -> V8. What are the neighbors of each node?


In [ ]:
for i in range(len(dag_gt[:,0])):
    indices = np.where(dag_gt[i,:] == 1)
    if len(indices)


In [44]:
import re
import numpy as np
pattern = r'V\d+'
pattern_bt = r'bt\d'
bt_id = 0
adj = np.zeros((10,10))

adj_ls = []
# Open the file in read mode
with open('response.txt', 'r') as file:
    # Iterate over each line in the file
    for line in file:
        # Process the line (strip is used to remove the newline character)
        text = line.strip()
        line = re.sub(r'[^a-zA-Z0-9]', '', text)

        # Find all matches in the text
        bt_ind = re.findall(pattern_bt, line)

        # Print the extracted matches
        
        if len(bt_ind) > 0:
            adj_ls.append(adj)
            adj = np.zeros((10,10))
            bt_id += 1
        else:
            neighbors = re.findall(pattern, line)
            for i in range(len(neighbors)):
                if i == 0:
                    ni = re.sub(r'[^0-9]', '', neighbors[i])
                    row_index = int(ni)
                else: 
                    ni = re.sub(r'[^0-9]', '', neighbors[i])
                    col_index = int(ni)
                    adj[row_index,col_index]  = 1
adj_ls.append(adj)
    

In [45]:
bt_id,len(adj_ls)

(5, 6)